In [1]:
# Dependencies and Setup
#%matplotlib notebook
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sts
from pprint import pprint

import requests
import json
import time
import gmaps
#from us import states
from config import gkey

In [2]:
# Read the csv file containing the Bicycle Crash data for North Carolina
input_file_path = "Resources/bicycle-crash-data-chapel-hill-region.csv"
crash_data = pd.read_csv(input_file_path, delimiter = ";")

# View the csv file containing the Bicycle Crash data for North Carolina
crash_data

# There are 11266 records in 55 columns

,geo_point_2d,Ambulance,BikeAge,BikeAgeGrp,BikeAlcDrg,BikeAlcFlg,BikeDir,BikeInjury,BikePos,BikeRace,...,RdConditio,RdConfig,RdDefects,RdFeature,RdSurface,RuralUrban,SpeedLimit,TraffCntrl,Weather,Workzone
0,"33.9437020597,-78.3493385901",Yes,70,70+,No,No,With Traffic,C: Possible Injury,Travel Lane,White,...,Dry,"Two-Way, Not Divided",None,No Special Feature,Coarse Asphalt,Rural,40 - 45 MPH,No Control Present,Clear,No
1,"35.3217726,-79.0388423701",Yes,8,6-10,No,No,With Traffic,C: Possible Injury,Travel Lane,White,...,Dry,"Two-Way, Not Divided",None,T-Intersection,Smooth Asphalt,Rural,50 - 55 MPH,Stop Sign,Clear,No
2,"35.77105009,-78.6073184803",Yes,13,11-15,No,No,Facing Traffic,A: Suspected Serious Injury,Travel Lane,Black,...,Dry,"Two-Way, Divided, Positive Median Barrier",None,No Special Feature,Coarse Asphalt,Urban,30 - 35 MPH,Stop And Go Signal,Cloudy,No
3,"35.9394515202,-77.7953706304",Yes,52,50-59,No,No,Facing Traffic,C: Possible Injury,Travel Lane,Black,...,Dry,"One-Way, Not Divided",None,Four-Way Intersection,Coarse Asphalt,Urban,30 - 35 MPH,Stop And Go Signal,Clear,No
4,"36.0927628197,-80.2653764398",Yes,27,25-29,No,No,Not Applicable,C: Possible Injury,Non-Roadway,White,...,Dry,"Two-Way, Not Divided",None,No Special Feature,Smooth Asphalt,Urban,30 - 35 MPH,"Double Yellow Line, No Passing Zone",Clear,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11261,"34.2546581498,-77.9197554903",Yes,26,25-29,No,No,Facing Traffic,C: Possible Injury,Travel Lane,White,...,Dry,"Two-Way, Not Divided",None,No Special Feature,Coarse Asphalt,Urban,30 - 35 MPH,Stop And Go Signal,Clear,No
11262,"35.2123611203,-80.7913412696",Yes,20,20-24,No,No,Facing Traffic,C: Possible Injury,Sidewalk / Crosswalk / Driveway Crossing,Black,...,Dry,"Two-Way, Not Divided",None,No Special Feature,Smooth Asphalt,Urban,30 - 35 MPH,No Control Present,Cloudy,No
11263,"35.64331907,-80.4828223001",Yes,11,11-15,No,No,Facing Traffic,C: Possible Injury,Travel Lane,Unknown/Missing,...,Dry,"Two-Way, Divided, Unprotected Median",None,No Special Feature,Smooth Asphalt,Urban,40 - 45 MPH,No Control Present,Clear,No
11264,"34.1935523204,-77.9356315301",No,60,60-69,No,No,With Traffic,C: Possible Injury,Travel Lane,White,...,Dry,"Two-Way, Not Divided",None,No Special Feature,Coarse Asphalt,Urban,40 - 45 MPH,No Control Present,Clear,No


In [3]:
# Cleaning Crash Group
crash_data["CrashGrp"] = crash_data["CrashGrp"].replace(
    {"Bicyclist Failed to Yield—Midblock": "Bicyclist Failed to Yield - Midblock",
    "Bicyclist Failed to Yield—Sign-Controlled Intersection": "Bicyclist Failed to Yield - Sign-Controlled Intersection",
    "Bicyclist Failed to Yield—Signalized Intersection": "Bicyclist Failed to Yield - Signalized Intersection",
    "Bicyclist Left Turn/Merge": "Bicyclist Left Turn / Merge",
    "Bicyclist Right Turn/Merge": "Bicyclist Right Turn / Merge",
    "Crossing Paths—Other Circumstances": "Crossing Paths - Other Circumstances",
    "Loss of Control/Turning Error": "Loss of Control / Turning Error",
    "Motorist Failed to Yield—Midblock": "Motorist Failed to Yield - Midblock",
    "Motorist Failed to Yield—Sign-Controlled Intersection": "Motorist Failed to Yield - Sign-Controlled Intersection",
    "Motorist Failed to Yield—Signalized Intersection": "Motorist Failed to Yield - Signalized Intersection",
    "Motorist Left Turn/Merge": "Motorist Left Turn / Merge",
     "Motorist Right Turn/Merge": "Motorist Right Turn / Merge",
     "Nonroadway": "Non-Roadway",
     "Other/Unknown—Insufficient Details": "Other / Unknown - Insufficient Details",
     "Other/Unusual Circumstances": "Other / Unusual Circumstances",
     "Parallel Paths—Other Circumstances": "Parallel Paths - Other Circumstances"
    })

# Check if all values were replaced correctly
# Crashes by Crash Gpr
crashes_by_crashgrp = crash_data.groupby(["CrashGrp"])["geo_point_2d"].count()
crashes_by_crashgrp.sort_values(ascending=False)

CrashGrp
Motorist Overtaking Bicyclist                               2207
Motorist Failed to Yield - Sign-Controlled Intersection     1073
Motorist Left Turn / Merge                                   978
Motorist Failed to Yield - Midblock                          824
Bicyclist Failed to Yield - Midblock                         739
Bicyclist Failed to Yield - Sign-Controlled Intersection     679
Crossing Paths - Other Circumstances                         674
Motorist Right Turn / Merge                                  585
Bicyclist Failed to Yield - Signalized Intersection          530
Non-Roadway                                                  503
Bicyclist Left Turn / Merge                                  503
Loss of Control / Turning Error                              477
Motorist Failed to Yield - Signalized Intersection           373
Head-On                                                      300
Bicyclist Overtaking Motorist                                232
Parallel Paths -

In [5]:
# Number of cities included in the study
total_crash_count = crash_data["geo_point_2d"].count()
county_count = crash_data["County"].nunique()
city_count = crash_data["City"].nunique()
print(f"Total number of Bicycle Crashes in North Carolina (2007-2018) = {total_crash_count}")
print (f"Number of Counties included in the study: {county_count}")
print (f"Number of Cities included in the study: {city_count}")

Total number of Bicycle Crashes in North Carolina (2007-2018) = 11266
Number of Counties included in the study: 100
Number of Cities included in the study: 327
